In [ ]:
# Standard libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import IntProgress
from IPython.display import display

# Append base directory
import os,sys,inspect
rootname = "pub-2020-exploratory-analysis"
thispath = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
rootpath = os.path.join(thispath[:thispath.index(rootname)], rootname)
sys.path.append(rootpath)
print("Appended root directory", rootpath)

from mesostat.metric.metric import MetricCalculator
from mesostat.utils.qt_helper import gui_fname, gui_fnames, gui_fpath
from mesostat.utils.hdf5_io import DataStorage

from lib.sych.data_fc_db_raw import DataFCDatabase
import lib.analysis.pid as pid

%load_ext autoreload
%autoreload 2

In [ ]:
# tmp_path = root_path_data if 'root_path_data' in locals() else "./"
params = {}
# params['root_path_data'] = './'
params['root_path_data'] = '/media/alyosha/Data/TE_data/yarodata/sych_preprocessed'
# params['root_path_data'] = '/media/aleksejs/DataHDD/work/data/yaro/neuronal-raw-pooled'
# params['root_path_data'] = gui_fpath('h5path', './')

In [ ]:
dataDB = DataFCDatabase(params)

In [ ]:
h5outname = 'sych_result_higher_order_df.h5'

In [ ]:
mc = MetricCalculator(serial=False, verbose=False, nCore=4)

## All - Distribution - Nosession

In [ ]:
import lib.analysis.pid_joint as pid_joint

In [ ]:
pwdAllH5_2 = '/media/alyosha/Data/TE_data/yarodata/sych_preprocessed/sych_result_multiregional_pid_all_df.h5'
pwdAllH5_2_Rand = '/media/alyosha/Data/TE_data/yarodata/sych_preprocessed/sych_result_multiregional_pid_all_df_rand.h5'

In [ ]:
dfSummary = pid_joint.pid_all_summary_df(pwdAllH5_2)

In [ ]:
dfSummaryRand = pid_joint.pid_all_summary_df(pwdAllH5_2_Rand, parserName='Rand')

In [ ]:
pid_joint.cdfplot(pwdAllH5_2, dfSummary)

In [ ]:
pid_joint.test_avg_bits(dataDB, mc, pwdAllH5_2, dfSummary)

In [ ]:
pid_joint.scatter_effsize_bits(pwdAllH5_2, dfSummary)

In [ ]:
pid_joint.plot_triplets(dataDB, pwdAllH5_2, dfSummary, nTop=20)

In [ ]:
pid_joint.plot_singlets(dataDB, pwdAllH5_2, dfSummary, nTop=20)

In [ ]:
# TODO: Unique: Sweep over pairs of (S1,T) vs S2, as opposed to (S1,S2) vs T
pid_joint.plot_2D_avg(dataDB, pwdAllH5_2, dfSummary, dropChannels=None, avgAxis=1)

In [ ]:
pid_joint.plot_2D_bytarget(dataDB, pwdAllH5_2, dfSummary, 'Rt', dropChannels=None)

In [ ]:
pid_joint.plot_2D_bytarget_synergy_cluster(dataDB, pwdAllH5_2, dfSummary, 'VPL',
                                           dropChannels=[21], clusterParam=2, dropWeakChannelThr=0.1)

In [ ]:
pid_joint.plot_unique_top_pairs(dataDB, pwdAllH5_2, dfSummary, nTop=20, dropChannels=None)

# Consistency

## 1. Across Mice

In [ ]:
pid_joint.plot_consistency_bymouse(pwdAllH5_2, dfSummary, dropChannels=[21], performance='naive',
                                   kind='fisher', limits=[0, 1])
pid_joint.plot_consistency_bymouse(pwdAllH5_2, dfSummary, dropChannels=[21], performance='expert',
                                   kind='fisher', limits=[0, 1])

In [ ]:
pid_joint.plot_consistency_byphase(pwdAllH5_2, dfSummary, dropChannels=[21], performance='naive',
                                   kind='fisher', limits=[0, 1], datatype='bn_trial')
pid_joint.plot_consistency_byphase(pwdAllH5_2, dfSummary, dropChannels=[21], performance='expert',
                                   kind='fisher', limits=[0, 1], datatype='bn_trial')
pid_joint.plot_consistency_byphase(pwdAllH5_2, dfSummary, dropChannels=[21], performance='naive',
                                   kind='fisher', limits=[0, 1], datatype='bn_session')
pid_joint.plot_consistency_byphase(pwdAllH5_2, dfSummary, dropChannels=[21], performance='expert',
                                   kind='fisher', limits=[0, 1], datatype='bn_session')

In [ ]:
pid_joint.plot_consistency_bytrialtype(pwdAllH5_2, dfSummary, dropChannels=[21], performance='naive', datatype='bn_trial',
                                 trialTypes=['iGO', 'iNOGO'], kind='fisher', fisherThr=0.1, limits=[0, 1])
pid_joint.plot_consistency_bytrialtype(pwdAllH5_2, dfSummary, dropChannels=[21], performance='expert', datatype='bn_trial',
                                 trialTypes=['iGO', 'iNOGO'], kind='fisher', fisherThr=0.1, limits=[0, 1])
pid_joint.plot_consistency_bytrialtype(pwdAllH5_2, dfSummary, dropChannels=[21], performance='naive', datatype='bn_session',
                                 trialTypes=['iGO', 'iNOGO'], kind='fisher', fisherThr=0.1, limits=[0, 1])
pid_joint.plot_consistency_bytrialtype(pwdAllH5_2, dfSummary, dropChannels=[21], performance='expert', datatype='bn_session',
                                 trialTypes=['iGO', 'iNOGO'], kind='fisher', fisherThr=0.1, limits=[0, 1])

## Sanity check if PID matches pre-computed

In [ ]:
for datatype in ['bn_session', 'bn_trial']:
    for intervName in intervNames:
        for trialType in [None, 'iGO', 'iNOGO']:
            for mousename in ['mvg_4']: #sorted(dataDB.mice):
                print(datatype, intervName, trialType, mousename)
                
                channelNames = dataDB.get_channel_labels(mousename)
                dataLst = dataDB.get_neuro_data({'mousename': mousename}, datatype=datatype,
                                                trialType=trialType,
                                                zscoreDim='rs', intervName=intervName)

                display(pid.pid(dataLst, mc, channelNames, ['S1_bf', 'VPL'], ['LGP'], nPerm=100, nBin=4))
                break
            break
        break
    break